# Used Car Price Prediction

[Kaggle Resource](https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset)

After countless total hours... probably even total days of learning, studying, preparation, examination of best practices, and relearning... it's finally time to get down to business.

I've also recently decided that you can only do so much preparation for a complicated field like ML, so I'm going to likely have to fill in some gaps along the way here. Thankfully, Kaggle has a LOT of publicly posted code to assist me here. I will abstain from referencing it as long as I can though, and will make sure to give credit where it's due.

Anyway, I believe this data marks a perfect balance between a challenging and approachable project. Let's begin.

# Load and Examine Data

In [537]:
import pandas as pd
import re
import numpy as np
import sklearn as skl
import seaborn as sns
import matplotlib.pyplot as plt

In [538]:
cars = pd.read_csv("data/used_cars.csv")
print(len(cars))

4009


Let's just get a quick overview of some basic precleaning stuff.

In [539]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [540]:
print(f"Missing Values:\n{cars.isna().sum()}\n\nTotal Duplicates:\n{cars.duplicated().sum()}")

Missing Values:
brand             0
model             0
model_year        0
milage            0
fuel_type       170
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

Total Duplicates:
0


Now that that's out of the way, let's actually take a peak at the data now.

In [541]:
cars.head(10)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"
5,Acura,ILX 2.4L,2016,"136,397 mi.",Gasoline,2.4 Liter,F,Silver,Ebony.,None reported,NaN,"$14,798"
6,Audi,S3 2.0T Premium Plus,2017,"84,000 mi.",Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,Black,None reported,Yes,"$31,000"
7,BMW,740 iL,2001,"242,000 mi.",Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,Green,None reported,Yes,"$7,300"
8,Lexus,RC 350 F Sport,2021,"23,436 mi.",Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,Black,None reported,Yes,"$41,927"
9,Tesla,Model X Long Range Plus,2020,"34,000 mi.",NaN,534.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,"$69,950"


# Clean the Data

Now that the preliminary stuff is out of the way, let's start cleaning the data up.

It'd be best to start simple here and worry about the basic numeric conversion first. We'll start by removing unnecessary string values from columns that will be designated as strictly numeric. I also think it'd be interesting to see if there are any differences in the model outputs when using `float32` instead of `float64`. We'll get to that much later though.

For now, let's start with simple data cleansing. Then we can chip away at the more tricky regex stuff and increase the complexity as we get a feel for the data.

**Note:** We can definitely work on making this SIGNIFICANTLY cleaner by aggregating a lot of this into less lines of code. We want BEST PRACTICES AND THIS IS NOT THAT.

In [542]:
# Remove non integer string values
cars[["milage", "price"]] = cars[["milage", "price"]].replace(r'[^\d]', '', regex=True)    

# `engine` column
cars["horsepower"] = cars["engine"].str.extract(r'([\d.]+)HP', expand=False)
cars["liters"] = cars["engine"].str.extract(r'([\d.]+)L', expand=False)

# `fuel_type` column
cars["gas_diesel_other"] = (
    cars["fuel_type"].str.contains("gasoline", case=False, na=False) |
    cars["fuel_type"].str.contains("diesel", case=False, na=False) |
    cars["fuel_type"].str.contains("fuel", case=False, na=False)
)
cars["hybrid"] = cars["fuel_type"].str.contains("hybrid", case=False, na=False)
cars["electric"] = cars["fuel_type"].isna()
print(f"Conflicting fuel type values? {(cars[['gas_diesel_other', 'hybrid', 'electric']].sum(axis=1) > 1).any()}")

# `transmission` column
cars["automatic_transmission"] = (
    cars["transmission"].str.contains("A/T", case=False) |
    cars["transmission"].str.contains("AT", case=False) |
    cars["transmission"].str.contains("auto", case=False) |
    cars["transmission"].str.contains("automatic", case=False) |
    cars["transmission"].str.contains("cvt", case=False)
)
cars['dual_shift_transmission'] = cars['transmission'].str.contains("dual", case=False)
cars["manual_transmission"] = (
    cars['transmission'].str.contains("manual", case=False) |
    cars['transmission'].str.contains("m/t", case=False) |
    cars['transmission'].str.contains("mt", case=False)
)
cars.loc[cars['dual_shift_transmission'], ['automatic_transmission', 'manual_transmission']] = False
cars.loc[cars['manual_transmission'], ['automatic_transmission', 'dual_shift_transmission']] = False
mask = ~(cars[['automatic_transmission', 'dual_shift_transmission', 'manual_transmission']] == False).all(axis=1) # Remove rows with `False` in all three newly created transmission columns
cars = cars[mask]
print(f"Conflicting transmission values? {(cars[['automatic_transmission', 'manual_transmission', "dual_shift_transmission"]].sum(axis=1) > 1).any()}")

# `accident` and `clean_title` columns
cars["accident_bool"] = cars["accident"] != "None reported"
cars.dropna(subset=['accident'], inplace=True)

# We'll decide if we should drop the color columns a bit later. I'll have to investigate the correlations before deciding
cars.drop(["fuel_type", "engine", "transmission", "accident", "clean_title"], axis=1, inplace=True)
cars

Conflicting fuel type values? False
Conflicting transmission values? False


,brand,model,model_year,milage,ext_col,int_col,price,horsepower,liters,gas_diesel_other,hybrid,electric,automatic_transmission,dual_shift_transmission,manual_transmission,accident_bool
0,Ford,Utility Police Interceptor Base,2013,51000,Black,Black,10300,300.0,3.7,True,False,False,True,False,False,True
1,Hyundai,Palisade SEL,2021,34742,Moonlight Cloud,Gray,38005,NaN,3.8,True,False,False,True,False,False,True
2,Lexus,RX 350 RX 350,2022,22372,Blue,Black,54598,NaN,NaN,True,False,False,True,False,False,False
3,INFINITI,Q50 Hybrid Sport,2015,88900,Black,Black,15500,354.0,3.5,False,True,False,True,False,False,False
4,Audi,Q3 45 S line Premium Plus,2021,9835,Glacier White Metallic,Black,34999,NaN,2.0,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004,Bentley,Continental GT Speed,2023,714,C / C,Hotspur,349950,NaN,6.0,True,False,False,True,False,False,False
4005,Audi,S4 3.0T Premium Plus,2022,10900,Black,Black,53900,349.0,3.0,True,False,False,False,True,False,False
4006,Porsche,Taycan,2022,2116,Black,Black,90998,NaN,NaN,False,False,True,True,False,False,False
4007,Ford,F-150 Raptor,2020,33000,Blue,Black,62999,450.0,3.5,True,False,False,True,False,False,False


# Fill in Missing Values

Now that the more simple stuff is out of the way, let's start filling in those missing values. The way I see it we have two options: We can either take the mean values for the relevant column of their respective brand, or run some separate regressions to potentially get some more accurate values to fill in. Let's do both and see the results. The second option sounds a bit more interesting and potentially more useful... but I guess we'll have to see what we end up with first.

## Fill Missing Values with Brand Mean

Start simple: group by brand name, find the mean of `horsepower` and `liters` for the given brand, then insert values into NaN cells

## Fill Missing Values with Predictions

Now let's get a bit more advanced. Project the `horsepower` and `liters` values using linear regression. There are many ways to do this, so let's walk through a few different ideas:
*